In [6]:
import os 
import gym
import gym_super_mario_bros
from gym_super_mario_bros.actions import RIGHT_ONLY
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
from nes_py.wrappers import JoypadSpace
import tensorboard as tf

# Função para criar o ambiente
def make_env():
    env = gym_super_mario_bros.make('SuperMarioBros-v1')
    env = JoypadSpace(env, RIGHT_ONLY)
    env = gym.wrappers.GrayScaleObservation(env, keep_dim=True)
    env = CustomRescaleObservation(env, low=0, high=255)
    env = gym.wrappers.FrameStack(env, num_stack=6)
    return env

# Classe personalizada para redimensionar as observações
class CustomRescaleObservation(gym.ObservationWrapper):
    def __init__(self, env, low, high):
        super(CustomRescaleObservation, self).__init__(env)
        self.low = low
        self.high = high

    def observation(self, observation):
        return observation * (self.high - self.low) + self.low

# Callback personalizado para recompensas e penalidades
class CustomCallback(BaseCallback):
    def __init__(self, check_freq, save_path, verbose=1):
        super(CustomCallback, self).__init__(verbose)
        self.x_pos = None
        self.check_freq = check_freq
        self.save_path = save_path
        self.rewards = 0
        self.penalties = 0
        self.progress_bar = None

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        # Obtem informações sobre o ambiente após a etapa
        info = self.locals.get("info", {})

        # Recompensa o agente em 1 ponto se ele uma moeda
        if "coins" in info and info["coins"] > 0:
            self.rewards += 1
            self.logger.record("coin_reward", self.rewards)

        # Penaliza o agente em 15 pontos se morrer
        if "life" in info and info["life"] < 2:
            self.penalties += 15
            self.logger.record("death_penalty", self.penalties)

        # Verifica se o agente está parado na mesma posição
        if "x_pos" in info and info["x_pos"] == self.x_pos:
            # Penaliza o agente em 0.3 pontos se ficar em uma mesma posição
            self.penalties += 0.3
            self.logger.record("stagnation_penalty", self.penalties)
            # Verifiqua a altura do agente 
            if "y_pos" in info and info["y_pos"] > 200:
                # Recompensa o agente com 10 pontos ao executar um pulo com uma altura maior que 200
                self.rewards += 10
                self.logger.record("height_reward", 10)
            
            if "y_pos" in info and info["y_pos"] < 150:
                # Penaliza o agente com 0.3 pontos ao executar um pulo com uma altura menor que 150
                self.penalties += 0.3
                self.logger.record("stagnation_penalty", self.penalties)

        # Salva a posição atual para a próxima iteração
        self.x_pos = info.get("x_pos", None)
        self.y_pos = info.get("y_pos", None)

        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'melhor_modelo_mario_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

# Cria o ambiente do jogo
env = make_env()

# Diretórios onde seram salvos os modelos e logs
CHECKPOINT_DIR = './mario_model_v4/'
LOG_DIR = './logs_v2/'

# Crie o modelo PPO com a política MlpPolicy ou Use um já salvo
##1 model = PPO("MlpPolicy", env, verbose=1,tensorboard_log=LOG_DIR, batch_size=128, learning_rate=0.0005, vf_coef=0.5, ent_coef=0.01)
model = PPO.load('./mario_model_v3/melhor_modelo_mario_450000', env=env, tensorboard_log=LOG_DIR, batch_size=128, learning_rate=0.0005, vf_coef=0.5, ent_coef=0.01) ##2

# Declaração do treinamento com callback personalizado
custom_callback = CustomCallback(check_freq=50000, save_path=CHECKPOINT_DIR)
model.learn(total_timesteps=550000, callback=custom_callback)

# Salva o modelo depois de concluir
model.save("mario_model")


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./logs_v2/PPO_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.65e+03 |
|    ep_rew_mean     | 1.92e+03 |
| time/              |          |
|    fps             | 130      |
|    iterations      | 1        |
|    time_elapsed    | 15       |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1.61e+03     |
|    ep_rew_mean          | 2.16e+03     |
| time/                   |              |
|    fps                  | 48           |
|    iterations           | 2            |
|    time_elapsed         | 84           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0015531909 |
|    clip_fraction        | 0.0178       |
|    clip_range           | 0.2         